In [3]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np
import requests
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup 
from keras import layers
from keras import losses

In [3]:
import pandas as pd

#inputFile = open('/Users/jacobjohnson/data_sets/congressional_tweet_training_data.csv', 'r')

df = pd.read_csv('/Users/jacobjohnson/data_sets/congressional_tweet_training_data.csv', names=['favorite_count', 'full_text', 'hashtags', 'retweet_count', 'year', 'party_id'], 
                    dtype={'favorite_count': int, 'full_text': str, 'hashtags': str, 'retweet_count': int, 'year': int, 'party_id': str}, 
                    na_values='0', keep_default_na=False, skipinitialspace=True, skiprows=1, sep=',').fillna('', inplace=True)

# inputArray = inputFile.read()
# inputArray = inputArray.split()
# nestedInputArray = []
# nestedInputArray = [inputArray[i:i+2] for i in range(0, len(inputArray), 2)]
# nestedInputArray = [[float(x) for x in lst] for lst in nestedInputArray]
# for i in range(0, len(nestedInputArray)):
#     nestedInputArray[i].append(0)


ValueError: Integer column has NA values in column 0

In [18]:
import pandas as pd

train_df = pd.read_csv('/Users/jacobjohnson/data_sets/congressional_tweet_training_data.csv', names=['favorite_count', 'full_text', 'hashtags', 'retweet_count', 'year', 'party_id'], 
                    # dtype={'favorite_count': int, 'full_text': str, 'hashtags': str, 'retweet_count': int, 'year': int, 'party_id': str}, 
                    skipinitialspace=True, skiprows=1, sep=',')

test_df = pd.read_csv('/Users/jacobjohnson/data_sets/congressional_tweet_test_data.csv', names=['id', 'favorite_count', 'full_text', 'hashtags', 'retweet_count', 'year', 'party'], 
                    # dtype={'id': int, 'favorite_count': int, 'full_text': str, 'hashtags': str, 'retweet_count': int, 'year': int, 'party': str}, 
                    skipinitialspace=True, skiprows=1, sep=',')

train_df.pop('year')
test_df.pop('year')

print(train_df.iloc[1])
print(train_df.columns)

print(test_df.iloc[:5])

favorite_count                                                  258
full_text         b"Today I'm urging the @CDCgov to immediately ...
hashtags                                                Coronavirus
retweet_count                                                   111
party_id                                                          R
Name: 1, dtype: object
Index(['favorite_count', 'full_text', 'hashtags', 'retweet_count', 'party_id'], dtype='object')
   id  favorite_count                                          full_text  \
0   0              70  b'#TaxReform improved the playing field for Am...   
1   1              27  b'This #NativeWomensEqualPay Day, we recommit ...   
2   2              49  b"\xe2\x80\x9cI became convinced that our gene...   
3   3              14  b'During #NationalAdoptionMonth, we honor the ...   
4   4              13  b'Happy #AirborneDay to our @USArmy paratroope...   

                        hashtags  retweet_count party  
0                      TaxRe

In [5]:
import pandas as pd
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split


df_training = train_df

hashtags = df_training['hashtags'].values
y = df_training['party_id'].values

# ds_split, info = tfds.load('trainingData', split=['train[:20%]', 'train[20%:]'], as_supervised=True, with_info=True)


hashtags_train, hashtags_test, y_train, y_test = train_test_split(hashtags, y, test_size=0.25, random_state=1000)



/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import layers

np.set_printoptions(precision=3, suppress=True)

training_data = np.array(train_df)


In [7]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(hashtags_train)

X_train = vectorizer.transform(hashtags_train)
X_test  = vectorizer.transform(hashtags_test)
X_train

<643352x79894 sparse matrix of type '<class 'numpy.int64'>'
	with 948308 stored elements in Compressed Sparse Row format>

In [8]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy: ", score)

Accuracy:  0.7679190118022299


In [9]:
from keras.models import Sequential
from keras import layers

input_dim = X_train.shape[1]

# model = Sequential()
# model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
# model.add(layers.Dense(1, activation='sigmoid'))


model = Sequential([
  tf.keras.layers.Dense(10, input_dim=input_dim, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.relu),
  tf.keras.layers.Dense(2)
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                798950    
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 2)                 22        
                                                                 
Total params: 799,082
Trainable params: 799,082
Non-trainable params: 0
_________________________________________________________________


2022-05-04 15:02:18.453794: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
print(training_data)

[[0
  'b"RT @KUSINews: One of our longtime viewers was in Congressman @DarrellIssa\'s office today and sent us these awesome photos, he has a #KUSI c\\xe2\\x80\\xa6"'
  'KUSI' 10 2017.0 'R']
 [258
  'b"Today I\'m urging the @CDCgov to immediately launch a 24/7 phone hotline to address questions from Americans regarding the #Coronavirus.\\n\\nI\'m also urging the agency to hold regular calls with state &amp; local health officials to provide up-to-date info &amp; provide any resources needed. https://t.co/xRzNim8RHM"'
  'Coronavirus' 111 2020.0 'R']
 [0
  "b'Tomorrow, #MO03 seniors graduate from Calvary Lutheran, Ft. Zumwalt East, Ft. Zumwalt West, Ft. Zumwalt North and Ft. Zumwalt South (2/3)'"
  'MO03' 2 2014.0 'R']
 ...
 [1
  "b'Congratulations to the new #MissTeenUSA Logan West of Southington \\xe2\\x80\\x93 a first for #CT.'"
  'MissTeenUSA CT' 4 2012.0 'D']
 [2174
  "b'Speaking of dishonesty. Nothing like being caught on camera. #mosen https://t.co/StSwn71F5D'"
  'mosen' 1168 2018

In [13]:
input_tensor = tf.convert_to_tensor(training_data)
# # input_tensor = tf.constant(df)

# predictions = model(input_tensor)
# predictions[:5]
# tf.nn.softmax(predictions[:5])

# train_features = training_data.copy()
# train_labels = train_features['party_id']


# training_data.fit(train_features, train_labels, epochs=10)

input_tensor.fit(epochs=10)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).